In [ ]:
# Genrates an interactive 3D Plot and generate an HTML 
def three_D_multi_plot_factory(dataframe,
                               x_axis,
                               y_axis,
                               category_z,
                               quantiles,
                               aggfunc,
                               title = "",
                               x_title ="",
                               y_title = "",
                               z_title = ["",""],
                               margin={'r': 0, 'l': 0, 't': 40, 'b': 25},
                               imputer_n = 3,
                               html_name = "",
                               output_html = False,
                               auto_commit = False,
                               height = 1000):
    
    if len(category_z) > 3:
        cols = 2 
        rows = math.ceil(len(category_z)/2)
        specs = [[{'is_3d': True},{'is_3d': True}] for x in range(rows)]
    else:
        cols =  len(category_z)
        rows =  1
        specs = [[{'is_3d': True} for x in range(cols)]]
        
    fig = make_subplots(rows=rows, cols=cols,subplot_titles=([label_processor(x)for x in category_z]), specs= specs, shared_xaxes = True, shared_yaxes = True)

    for z_axis, index in zip(category_z, it.product(range(1, rows + 1), range(1, cols + 1))) :
        slice_df = dataframe[[x_axis] + [y_axis] + [z_axis]].copy()
        slice_df[x_axis] = pd.qcut(x = slice_df[x_axis], q = quantiles)
        slice_df[y_axis] = pd.qcut(x = slice_df[y_axis], q = quantiles)
        pivot_slice_df = slice_df.pivot_table(index = x_axis,columns = y_axis, values = z_axis, aggfunc = aggfunc)

        imputer = KNNImputer(n_neighbors=imputer_n)
        pivot_slice_df = pd.DataFrame(imputer.fit_transform(pivot_slice_df), columns = pivot_slice_df.columns, index = pivot_slice_df.index)
        
        fig.add_trace(go.Surface(
                      z  = pivot_slice_df.values,
                      showlegend= False,
                      showscale=False,
                      ),row = index[0], col = index[1])
            
    fig.update_layout(title=title, margin = margin, height = height, font = default_font)
    fig.update_layout(
    title={
        'text': title,
        'y':1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
    
    fw=go.FigureWidget(fig)
    
    camera = dict(up=dict(x=0.4, y=0.4, z=0.5),
                  eye=dict(x=-1.4, y=-1.4, z=1.2))
    
    # So akward, but works better than everything else I tried.
    with fw.batch_update():
        for x in range(1,len(category_z)+1):
            fw.layout["scene" + str(x)] = scene = dict(
                                              camera = camera,
                                              xaxis = dict(
                                                title = dict(text = x_title, font = dict(family =  'Franklin Gothic', size = 10, color = '#7f7f7f')),
                                                nticks = len(pivot_slice_df.columns.tolist()),
                                                tickvals = list(range(len(pivot_slice_df.columns.tolist()))), 
                                                ticktext= ["{:.1f}".format((x.left + x.right)/2) for x in pivot_slice_df.columns.tolist()],
                                                tickfont = dict(family =  'Arial', size = 10, color = '#7f7f7f')),
                                              yaxis = dict(
                                                title = dict(text = y_title, font = dict(family =  'Franklin Gothic', size = 10, color = '#7f7f7f')),
                                                nticks = len(pivot_slice_df.index.tolist()),
                                                tickvals = list(range(len(pivot_slice_df.index.tolist()))), 
                                                ticktext= ["{:.0f}".format((x.left + x.right)/2) for x in pivot_slice_df.index.tolist()],
                                                tickfont = dict(family =  'Arial', size = 10, color = '#7f7f7f')),
                                              zaxis_title = dict(text = z_title, font = dict(family =  'Franklin Gothic', size = 10, color = '#7f7f7f'))) 
              
    path = "../fierceeagle.github.io/_includes/"+ html_name+".html"           
    if output_html:
        print("Writing HTML to Repo...")
        fw.write_html(path)
        print("Ave Imperator, morituri te salutant")
        
        
        if auto_commit: 
            github_hook()
            
    return plot_map_inline(path)

In [ ]:
## Creates a Tree Plot using the dtreeviz libary, can automatically write the resulting SVG to a file and push the local repo,
#  however need to consider to change the width and height parameter of the svg

def tree_plot_factory(X,
                      y,
                      X_test,
                      y_test,
                      X_cols,
                      y_col,
                      clf,
                      reverse_dict,
                      store_tree = False,
                      path = "./../fierceeagle.github.io/_includes/",
                      file_name = "place_holder",
                      hook = False,
                      plot_test = True):
    
    y = y.apply(lambda x : reverse_dict[x])
    
    # Fitting the Data, for later visualization
    clf.fit(X,y)    
    
    if y_col == "Education":
        class3colors = ['#886e9e', '#2f0047', '#e5e4fd']
    else: # religion
        class3colors = ['#f5e4ce', '#be3980', '#5f80ca']
    
    if plot_test: 
        viz = dtreeviz(clf, X_test, y_test.apply(lambda x : reverse_dict[x]),
                       target_name= y_col,
                       feature_names= X_cols,
                       class_names= list(reverse_dict.keys()),
                       colors = {'classes':[ None,  # 0 classes
                                             None,  # 1 class
                                             ['#FEFEBB', '#a1dab4'],  # 2 classes
                                             class3colors,  # RELIGION OR EDUCATION
                                             ['#f3dee0', '#9d0000', '#d69495', '#ba4a4b'],  # INCOME
                                             ['#FEFEBB', '#D9E6F5', '#a1dab4', '#41b6c4', '#e6e6e6'],  # 5
                                             ['#f95312', '#009195', '#c10a03', '#aa755c', '#68d5d9', '#e6e6e6']]} # ETHNICITY
            )  
    else:
        viz = dtreeviz(clf, X, y,
                       target_name= y_col,
                       feature_names= X_cols,
                       class_names= list(reverse_dict.keys()),
                       colors = {'classes':[ None,  # 0 classes
                                             None,  # 1 class
                                             ['#FEFEBB', '#a1dab4'],  # 2 classes
                                             class3colors,  # RELIGION OR EDUCATION
                                             ['#f3dee0', '#9d0000', '#d69495', '#ba4a4b'],  # INCOME
                                             ['#FEFEBB', '#D9E6F5', '#a1dab4', '#41b6c4', '#e6e6e6'],  # 5
                                             ['#f95312', '#009195', '#c10a03', '#aa755c', '#68d5d9', '#e6e6e6']]} # ETHNICITY
                       #colors = dict(zip(["White Majority","Strong Asian Minority","Strong Black Minority","Ethically Diverse","Asian Majority","Black Majority"],['#e6e6e6', '#68d5d9', '#f95312', '#aa755c', '#009195', '#c10a03']))
                      ) 
        

    if store_tree:
        print("Storing SVG")
        viz.save(path + file_name + ".svg")
        print("For Website usage, open the final Image and remove the height parameter \n and set the width to 100% so it scales on the website")
    
    if hook: 
        github_hook()
        
    return viz

In [ ]:
msoa_social_sunburst = msoa_social.copy()
msoa_social_sunburst["total"] = "Total"
msoa_social_sunburst = msoa_social_sunburst.groupby(by = ["total","Ethnicity", "Education", "Income"]).count().reset_index()
msoa_social_sunburst = msoa_social_sunburst.dropna()
#msoa_social_sunburst.sort_values(["Ethnicity","Education","Income"],ascending=False,inplace=True)
fig = px.sunburst(msoa_social_sunburst, 
                  path=["total","Ethnicity", "Education", "Income"], 
                  values='name',
                  branchvalues="total",
                  color_discrete_sequence = ['#e6e6e6','#f95312','#009195','#68d5d9','#c10a03','#aa755c']) 
fig.update_traces(insidetextorientation='radial', textfont = {'family': 'Franklin Gothic', "size" : 10})
path = "../fierceeagle.github.io/_includes/"+"Sunburst_"+".html"
print("Writing HTML to Repo...")
fig.write_html(path)
plot_map_inline(path)